In [ ]:
using Catalyst, DifferentialEquations, Distributions, CairoMakie
using DataFrames
using DelimitedFiles

In [ ]:
CN_distribution_low = truncated(Poisson(0.3), lower=1.0)

support = range(0,5)
plot(support, pdf.(CN_distribution_low, support); axis=(; title="Copy number distribution, MOI=0.3", xlabel="CN", ylabel="P[CN]"))

In [ ]:
CN_distribution_med = truncated(Poisson(3), lower=1.0)

support = range(0,10)
plot(support, pdf.(CN_distribution_med, support); axis=(; title="Copy number distribution, MOI=3", xlabel="CN", ylabel="P[CN]"))

In [ ]:
CN_distribution_high = truncated(Poisson(10), lower=1.0)

support = range(0,20)
plot(support, pdf.(CN_distribution_high, support); axis=(; title="Copy number distribution, MOI=10", xlabel="CN", ylabel="P[CN]"))

## Single transcript

In [ ]:
single_iFFL_rn = @reaction_network single_transcript_iFFL begin
    @parameters begin
        α_im=4.67e-2 #Transcription rate constant [1/s]
        δ_im=2.88e-4 #Immature mRNA degradation rate constant [1/s]
        δ_m=2.88e-4 #Mature mRNA degradation rate constant [1/s]
        δ_mi=2.88e-4 #miRNA degradation rate constant [1/s]
        δ_p=9.67e-5 #Protein degradation rate constant [1/s] 
        α_p=3.33e-4 #Translation rate constant [1/s]
        ζ=0.0 #Translation efficiency for RISC complex [unitless], Range: 0 to 1
        r_splicing=2.0e-3 #Splicing rate constant [1/s]
        r_drosha=1e-2 #pri-miRNA processing rate constant [1/s] 
        r_dicer=1e-3 #pre-miRNA processing rate constant [1/s]
        k_miRNA_bind=1e-5 #miRNA-RISC binding rate constant [1/mol*s]
        k_miRNA_deg=2.16e-5 #miRNA-RISC unbinding rate constant [1/s]
        k_mRNA_bind=1.84e-6 #RISC complex formation rate constant[1/mol*s]
        k_mRNA_unbind=0.303 #RISC complex deformation rate constant [1/s]
        k_deg=7e-3 #RISC-mediated mRNA degradation rate constant [1/s]
    end
    
    # Transcription
    α_im, gene --> immature_mRNA + gene #Set copy number as the initial number of "gene"
    α_im, gene --> ctl_mRNA + gene
    # Degradation of products
    δ_im, immature_mRNA --> ∅
    δ_m, mRNA --> ∅
    δ_m, ctl_mRNA --> ∅
    δ_mi, miRNA --> ∅
    δ_p, protein --> ∅
    δ_p, ctl_protein --> ∅
    # Production of protein
    α_p, mRNA --> protein + mRNA
    α_p, ctl_mRNA --> ctl_protein + ctl_mRNA
    α_p * ζ, risc_miRNA_mRNA --> protein + risc_miRNA_mRNA
    # Splicing
    r_splicing, immature_mRNA --> mRNA + pri_miRNA
    # miRNA processing
    r_drosha, pri_miRNA --> pre_miRNA # drosha
    r_dicer, pre_miRNA --> miRNA # dicer
    # Knockdown
    k_miRNA_bind, risc + miRNA --> risc_miRNA
    k_miRNA_deg, risc_miRNA --> risc
    (k_mRNA_bind, k_mRNA_unbind), risc_miRNA + mRNA <--> risc_miRNA_mRNA
    k_deg, risc_miRNA_mRNA --> risc_miRNA
end

In [ ]:
species(single_iFFL_rn)

In [ ]:
function prob_func_low(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_low), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function prob_func_med(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_med), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function prob_func_high(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_high), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function output_func(sol, i)
    [sol.u[end,:]], false
end

### Low moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_low), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(single_iFFL_rn, DiscreteProblem(single_iFFL_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_low, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_low_moi_st = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/single_transcript_10k_risc_low_moi.csv",eachrow(risc_10k_low_moi_st),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Single transcript iFFL: 10,000 RISC, 0.3 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 5000)
    )
s1 = scatter!(ax1, risc_10k_low_moi_st.copynum, risc_10k_low_moi_st.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_low_moi_st.copynum, risc_10k_low_moi_st.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 5000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_low_moi_st.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_low_moi_st.regulatedc, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/single_transcript_10000_risc_low_moi.svg",f)
current_figure()

### Medium moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_med), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(single_iFFL_rn, DiscreteProblem(single_iFFL_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_med, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_med_moi_st = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/single_transcript_10k_risc_med_moi.csv",eachrow(risc_10k_med_moi_st),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Single transcript iFFL: 10,000 RISC, 3 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 7000)
    )
s1 = scatter!(ax1, risc_10k_med_moi_st.copynum, risc_10k_med_moi_st.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_med_moi_st.copynum, risc_10k_med_moi_st.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 7000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_med_moi_st.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_med_moi_st.regulated, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/single_transcript_10000_risc_med_moi.svg",f)
current_figure()

### High moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_high), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(single_iFFL_rn, DiscreteProblem(single_iFFL_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_high, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_high_moi_st = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/single_transcript_10k_risc_high_moi.csv",eachrow(risc_10k_high_moi_st),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Single transcript iFFL: 10,000 RISC, 10 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 10000)
    )
s1 = scatter!(ax1, risc_10k_high_moi_st.copynum, risc_10k_high_moi_st.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_high_moi_st.copynum, risc_10k_high_moi_st.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 10000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_high_moi_st.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_high_moi_st.regulated, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/single_transcript_10000_risc_high_moi.svg",f)
current_figure()

## Dual transcript design

In [ ]:
double_iFFL_rn = @reaction_network double_transcript_iFFL begin
    # gene 1 is regulated gene w miRNA binding single_transcript_iFFL
    # gene 2 is gene containing miRNA intron
    @parameters begin
        α_im=4.67e-2 #Transcription rate constant gene 1 [1/s]
        δ_im=2.88e-4 #Immature mRNA degradation rate constant gene 2 [1/s]
        δ_m=2.88e-4 #Mature mRNA degradation rate constant gene 1 [1/s]
        δ_mi=2.88e-4 #miRNA degradation rate constant [1/s]
        δ_p=9.67e-5 #Protein degradation rate constant gene 1 [1/s] 
        α_p=3.33e-4 #Translation rate constant gene 1 [1/s]
        ζ=0.0 #Translation efficiency for RISC complex [unitless], Range: 0 to 1
        r_splicing=2.0e-3 #Splicing rate constant [1/s]
        r_drosha=1e-2 #pri-miRNA processing rate constant [1/s] 
        r_dicer=1e-3 #pre-miRNA processing rate constant [1/s]
        k_miRNA_bind=1e-5 #miRNA-RISC binding rate constant [1/mol*s]
        k_miRNA_deg=2.16e-5 #miRNA-RISC unbinding rate constant [1/s]
        k_mRNA_bind=1.84e-6 #RISC complex formation rate constant[1/mol*s]
        k_mRNA_unbind=0.303 #RISC complex deformation rate constant [1/s]
        k_deg=7e-3 #RISC-mediated mRNA degradation rate constant [1/s]
    end
    # Transcription
    α_im, gene --> mRNA + gene #Set copy number as the initial number of "gene1"
    α_im, gene --> ctl_mRNA + gene
    α_im, gene --> immature_mRNA + gene #Set copy number as the initial number of "gene2"
    # Degradation of products
    δ_im, immature_mRNA --> ∅
    δ_m, mRNA --> ∅
    δ_m, ctl_mRNA --> ∅
    δ_mi, miRNA --> ∅
    δ_p, protein --> ∅
    δ_p, ctl_protein --> ∅
    # Production of protein
    α_p, mRNA --> mRNA + protein
    α_p, ctl_mRNA --> ctl_protein + ctl_mRNA
    α_p * ζ, risc_miRNA_mRNA --> risc_miRNA_mRNA + protein
    # Splicing
    r_splicing, immature_mRNA --> pri_miRNA
    # miRNA processing
    r_drosha, pri_miRNA --> pre_miRNA # drosha
    r_dicer, pre_miRNA --> miRNA # dicer
    # Knockdown
    k_miRNA_bind, risc + miRNA --> risc_miRNA
    k_miRNA_deg, risc_miRNA --> risc
    (k_mRNA_bind, k_mRNA_unbind), risc_miRNA + mRNA <--> risc_miRNA_mRNA
    k_deg, risc_miRNA_mRNA --> risc_miRNA
end

In [ ]:
species(double_iFFL_rn)

In [ ]:
function prob_func_low(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_low), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function prob_func_med(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_med), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function prob_func_high(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_high), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function output_func(sol, i)
    [sol.u[end,:]], false
end

### Low moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_low), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(double_iFFL_rn, DiscreteProblem(double_iFFL_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_low, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_low_moi_dt = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/double_transcript_10k_risc_low_moi.csv",eachrow(risc_10k_low_moi_dt),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Double transcript iFFL: 10,000 RISC, 0.3 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 5000)
    )
s1 = scatter!(ax1, risc_10k_low_moi_dt.copynum, risc_10k_low_moi_dt.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_low_moi_dt.copynum, risc_10k_low_moi_dt.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 5000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_low_moi_dt.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_low_moi_dt.regulatec, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/double_transcript_10000_risc_low_moi.svg",f)
current_figure()

### Medium moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_med), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(double_iFFL_rn, DiscreteProblem(double_iFFL_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_med, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_med_moi_dt = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/double_transcript_10k_risc_med_moi.csv",eachrow(risc_10k_med_moi_dt),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Double transcript iFFL: 10,000 RISC, 3 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 7000)
    )
s1 = scatter!(ax1, risc_10k_med_moi_dt.copynum, risc_10k_med_moi_dt.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_med_moi_dt.copynum, risc_10k_med_moi_dt.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 7000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_med_moi_dt.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_med_moi_dt.regulatedc, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/double_transcript_10000_risc_med_moi.svg",f)
current_figure()

### High moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_high), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(double_iFFL_rn, DiscreteProblem(double_iFFL_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_high, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=1000)

In [ ]:
reg_vec = zeros(1000)
unreg_vec = zeros(1000)
cn_vec = zeros(1000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_high_moi_dt = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/double_transcript_10k_risc_high_moi.csv",eachrow(risc_10k_high_moi_dt),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Double transcript iFFL: 10,000 RISC, 10 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 10000)
    )
s1 = scatter!(ax1, risc_10k_high_moi_dt.copynum, risc_10k_high_moi_dt.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_high_moi_dt.copynum, risc_10k_high_moi_dt.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 10000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_high_moi_dt.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_high_moi_dt.regulated, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/double_transcript_10000_risc_high_moi.svg",f)
current_figure()

## Dual vector design

In [ ]:
double_vector_rn = @reaction_network double_transcript_iFFL begin
    # gene 1 is regulated gene w miRNA binding single_transcript_iFFL
    # gene 2 is gene containing miRNA intron
    @parameters begin
        α_im=4.67e-2 #Transcription rate constant gene 1 [1/s]
        δ_im=2.88e-4 #Immature mRNA degradation rate constant gene 2 [1/s]
        δ_m=2.88e-4 #Mature mRNA degradation rate constant gene 1 [1/s]
        δ_mi=2.88e-4 #miRNA degradation rate constant [1/s]
        δ_p=9.67e-5 #Protein degradation rate constant gene 1 [1/s] 
        α_p=3.33e-4 #Translation rate constant gene 1 [1/s]
        ζ=0.0 #Translation efficiency for RISC complex [unitless], Range: 0 to 1
        r_splicing=2.0e-3 #Splicing rate constant [1/s]
        r_drosha=1e-2 #pri-miRNA processing rate constant [1/s] 
        r_dicer=1e-3 #pre-miRNA processing rate constant [1/s]
        k_miRNA_bind=1e-5 #miRNA-RISC binding rate constant [1/mol*s]
        k_miRNA_deg=2.16e-5 #miRNA-RISC unbinding rate constant [1/s]
        k_mRNA_bind=1.84e-6 #RISC complex formation rate constant[1/mol*s]
        k_mRNA_unbind=0.303 #RISC complex deformation rate constant [1/s]
        k_deg=7e-3 #RISC-mediated mRNA degradation rate constant [1/s]
    end
    # Transcription
    α_im, gene --> mRNA + gene #Set copy number as the initial number of "gene1"
    α_im, gene --> ctl_mRNA + gene
    α_im, gene_miR --> immature_mRNA + gene_miR #Set copy number as the initial number of "gene2"
    # Degradation of products
    δ_im, immature_mRNA --> ∅
    δ_m, mRNA --> ∅
    δ_m, ctl_mRNA --> ∅
    δ_mi, miRNA --> ∅
    δ_p, protein --> ∅
    δ_p, ctl_protein --> ∅
    # Production of protein
    α_p, mRNA --> mRNA + protein
    α_p, ctl_mRNA --> ctl_protein + ctl_mRNA
    α_p * ζ, risc_miRNA_mRNA --> risc_miRNA_mRNA + protein
    # Splicing
    r_splicing, immature_mRNA --> pri_miRNA
    # miRNA processing
    r_drosha, pri_miRNA --> pre_miRNA # drosha
    r_dicer, pre_miRNA --> miRNA # dicer
    # Knockdown
    k_miRNA_bind, risc + miRNA --> risc_miRNA
    k_miRNA_deg, risc_miRNA --> risc
    (k_mRNA_bind, k_mRNA_unbind), risc_miRNA + mRNA <--> risc_miRNA_mRNA
    k_deg, risc_miRNA_mRNA --> risc_miRNA
end

In [ ]:
species(double_vector_rn)

In [ ]:
function prob_func_low(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_low), :gene_miR => rand(CN_distribution_low), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function prob_func_med(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_med), :gene_miR => rand(CN_distribution_med), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function prob_func_high(prob,i,repeat)
    remake(prob, u0 = [:gene => rand(CN_distribution_high), :gene_miR => rand(CN_distribution_high), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function output_func(sol, i)
    [sol.u[end,:]], false
end

### Low moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_low), :gene_miR => rand(CN_distribution_low), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(double_vector_rn, DiscreteProblem(double_vector_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_low, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][7]
    unreg_vec[j] = solns[j][1][1][8]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_low_moi_dv = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/double_vector_10k_risc_low_moi.csv",eachrow(risc_10k_low_moi_dv),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Double vector iFFL: 10,000 RISC, 0.3 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 5000)
    )
s1 = scatter!(ax1, risc_10k_low_moi_dv.copynum, risc_10k_low_moi_dv.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_low_moi_dv.copynum, risc_10k_low_moi_dv.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 5000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_low_moi_dv.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_low_moi_dv.regulated, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/double_vector_10000_risc_low_moi.svg",f)
current_figure()

### Medium moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_med), :gene_miR => rand(CN_distribution_med), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(double_vector_rn, DiscreteProblem(double_vector_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_med, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=10000)

In [ ]:
reg_vec = zeros(10000)
unreg_vec = zeros(10000)
cn_vec = zeros(10000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][7]
    unreg_vec[j] = solns[j][1][1][8]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_med_moi_dv = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/double_vector_10k_risc_med_moi.csv",eachrow(risc_10k_med_moi_dv),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Double vector iFFL: 10,000 RISC, 3 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 7000)
    )
s1 = scatter!(ax1, risc_10k_med_moi_dv.copynum, risc_10k_med_moi_dv.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_med_moi_dv.copynum, risc_10k_med_moi_dv.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 7000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_med_moi_dv.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_med_moi_dv.regulated, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/double_vector_10000_risc_med_moi.svg",f)
current_figure()

### High moi simulations

In [ ]:
u0 = [:gene => rand(CN_distribution_high), :gene_miR => rand(CN_distribution_high), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(double_vector_rn, DiscreteProblem(double_vector_rn, u0, tspan), SortingDirect(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func_high, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=1000)

In [ ]:
reg_vec = zeros(1000)
unreg_vec = zeros(1000)
cn_vec = zeros(1000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][7]
    unreg_vec[j] = solns[j][1][1][8]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)
risc_10k_high_moi_dv = DataFrame(regulated=reg_vec, unregulated=unreg_vec, copynum = cn_vec)
writedlm("jl_output/2024.05.09_modeling/double_vector_10k_risc_high_moi.csv",eachrow(risc_10k_high_moi_dv),",")

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Double vector iFFL: 10,000 RISC, 10 MOI", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, 0, 10000)
    )
s1 = scatter!(ax1, risc_10k_high_moi_dv.copynum, risc_10k_high_moi_dv.unregulated, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, risc_10k_high_moi_dv.copynum, risc_10k_high_moi_dv.regulated, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 10000))
linkyaxes!(ax1,ax2)
d1 = density!(risc_10k_high_moi_dv.unregulated, color=(:gray, 0.5), direction = :y)
d2 = density!(risc_10k_high_moi_dv.regulated, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.09_modeling/double_vector_10000_risc_high_moi.svg",f)
current_figure()